# Cuaderno 12: Multi-índices

El uso de multi-índices en objetos del tipo `Series` o `DataFrame` es útil para organizar jerárquicamente la información y puede considerarse como el equivalente a la definición de arreglos multdimensionales en `numpy`.

Examinaremos en este cuaderno algunas maneras para crear, utilizar y manipular multi-índices. Empezamos por importar los módulos de `pandas`y `numpy`:

In [ ]:
# importar pandas y NumPy
import numpy as np
import pandas as pd

Supongamos que queremos definir una serie de datos con el número de fallecidos en las provincias de Pichincha, Guayas y Manabí durante los meses de abril y mayo del año 2020. Una primera idea puede ser utilizar tuplas para indexar esta serie:

In [ ]:
# fallecidos en Pichincha, Guayas y Manabí en abril y mayo de 2020
# datos obtenidos de la base del Registro Civil
sfallecidos = pd.Series([12242, 2580, 1284, 1582, 1654, 1257], 
                        index= [('Guayas', 'abril'), ('Guayas', 'mayo'), 
                                ('Pichincha', 'abril'), ('Pichincha', 'mayo'), 
                                ('Manabí', 'abril'), ('Manabí', 'mayo')], 
                        name='Fallecidos')
print(sfallecidos)
print('---')
# mostrar los datos de Pichincha
print(sfallecidos[('Pichincha', 'abril'):('Pichincha', 'mayo')])
print('---')
# mostrar los datos de mayo
print(sfallecidos.index)
print([i for i in sfallecidos.index if i[1]=='mayo'])
print(sfallecidos[[i for i in sfallecidos.index if i[1]=='mayo']])


Sin embargo, esta opción es poco flexible. En particular, realizar consultas como la última a través de *list comprehensions* es muy ineficiente. Aquí es más conveniente definir un multi-índice:

In [ ]:
indice = pd.MultiIndex.from_tuples([('Guayas', 'abril'), ('Guayas', 'mayo'), 
                       ('Pichincha', 'abril'), ('Pichincha', 'mayo'), 
                       ('Manabí', 'abril'), ('Manabí', 'mayo')])
print(indice)

Puede fijarse este nuevo índice como índice de la serie `sfallecidos`, empleando el método `reindex`:

In [ ]:
sfallecidos = sfallecidos.reindex(indice)
print(sfallecidos)

Podemos trabajar con una serie o un DataFrame indexado por un multi-índice tal y como lo haríamos con un arreglo multidimensional en `numpy`:

In [ ]:
# mostrar los datos de Pichincha
print(sfallecidos['Pichincha',:])
print('---')
# mostrar los datos de mayo
print(sfallecidos[:,'mayo'])

Notar que en este caso, las dos dimensiones del multi-índice de la serie podrían haberse utlizado también como dimensiones para las filas y columnas de un DataFrame. De hecho, la transformación de una serie con un índice bidimensional en un DataFrame puede realizarse con el método `unstack()`:  

In [ ]:
dffallecidos = sfallecidos.unstack()
display(dffallecidos)

El método `stack()` realiza la operación inversa:

In [ ]:
print(dffallecidos.stack())

Sin embargo, los multi-índices son más generales: pueden emplearse multiíndices de más de dos dimensiones tanto en objetos tipo `Series` como en objetos tipo `DataFrame`. En estos últimos, pueden emplearse multi-índices tanto en las filas como en las columnas:

In [ ]:
sfallecidos2019 = pd.Series([1863, 1705, 1002, 1074, 554, 450], 
                        index= indice, 
                        name='Fallecidos 2019')
print(sfallecidos2019)
print('---')
df_fallecidos = pd.DataFrame({'2019' : sfallecidos2019, '2020' : sfallecidos})
display(df_fallecidos)

## Creación de multi-índices:

La forma más directa para crear un multi-índice es empleando uno de los métodos constructores disponibles en la clase `MultiIndex`. Por ejemplo, el método `from_tuples` empleado en la clase anterior, crea un multi-índice a partir de una lista de tuplas de la misma dimensión:

In [ ]:
# crear un muli-índice a partir de una lista de tuplas:
# los valores de cada componente de la tupla determinan los valores para cada nivel del índice
indice = pd.MultiIndex.from_tuples([('Guayas', 'abril'), ('Guayas', 'mayo'), 
                       ('Pichincha', 'abril'), ('Pichincha', 'mayo'), 
                       ('Manabí', 'abril'), ('Manabí', 'mayo')])
print(indice)

A veces resulta útil especificar nombres para los distintos niveles del índice. Esto puede hacerse con el parámetro `names`:

In [ ]:
indice = pd.MultiIndex.from_tuples([('Guayas', 'abril'), ('Guayas', 'mayo'), 
                       ('Pichincha', 'abril'), ('Pichincha', 'mayo'), 
                       ('Manabí', 'abril'), ('Manabí', 'mayo')], names=['Provincia', 'Mes'])
print(indice)
print('---')
sfallecidos = sfallecidos.reindex(indice)
print(sfallecidos)

El constructor `from_array` recibe como parámetro una lista de arreglos con los valores de cada uno de los niveles del multi-índice:

In [ ]:
indice = pd.MultiIndex.from_arrays([['Guayas', 'Guayas', 'Pichincha', 'Pichincha', 'Manabí', 'Manabí'], 
                                   ['abril', 'mayo', 'abril', 'mayo','abril', 'mayo']], 
                                  names=['Provincia', 'Mes'])
print(indice)


El constructor `from_product` genera las tuplas del multi-índice a partir del producto cartesiano de varias listas:

In [ ]:
indice = pd.MultiIndex.from_product([['Guayas', 'Pichincha', 'Manabí'], ['abril', 'mayo']], 
                                  names=['Provincia', 'Mes'])
print(indice)


En un DataFrame, pueden usarse multi-índices tanto para las filas (al fijar el parámetro `index` del constructor) como para las columnas (al fijar el parámetro `columns`). Por ejemplo, vamos a resumir en un DataFrame el número de nacimientos y de fallecimientos registrados en los meses de abril y mayo de los años 2019 y 2020:

In [ ]:
# crear multi-índice para usar en las filas
indice = pd.MultiIndex.from_product([['Guayas', 'Pichincha', 'Manabí'], ['abril', 'mayo']], 
                                  names=['Provincia', 'Mes'])
# crear multi-índice para usar en las columnas
columnas = pd.MultiIndex.from_product([['2019', '2020'], ['Nacimientos', 'Fallecimientos']], 
                                      names=['Año', 'Indicador'])
# crear DataFrame con datos inicializados a cero
df = pd.DataFrame(np.zeros((6,4), dtype=int), index= indice, columns= columnas)
display(df)
# llenar datos por columnas
df['2019', 'Nacimientos'] = [6190, 6175, 3975, 4003, 2342, 2327]  # datos ficticios
df['2019', 'Fallecimientos'] = [1863, 1705, 1002, 1074, 554, 450]
df['2020', 'Nacimientos'] = [6201, 6187, 3960, 3991, 2353, 2336]  # datos ficticios
df['2020', 'Fallecimientos'] = [12242, 2580, 1284, 1582, 1654, 1257]
display(df)

Podemos ahora acceder de manera fácil a los datos de un año:

In [ ]:
# datos del 2019
display(df['2019'])


## Indexación y selección

### Series con multi-índices

Consideremos una vez más la serie con los fallecimientos en abril y mayo de 2020, en las provincias de Guayas, Pichincha y Manabí.

In [ ]:
indice = pd.MultiIndex.from_arrays([['Guayas', 'Manabí', 'Pichincha', 'Pichincha', 'Manabí', 'Guayas'], 
                                   ['abril', 'mayo', 'abril', 'mayo','abril', 'mayo']], 
                                  names=['Provincia', 'Mes'])
sfallecidos = pd.Series([12242, 1257, 1284, 1582, 1654, 2580], 
                        index= indice, name='Fallecidos')
print(sfallecidos)


Para acceder a un elemento específico, utilizamos la tupla de valores correspondientes del multi-índice (los paréntesis pueden omitirse):

In [ ]:
# número de fallecidos en mayo en Pichincha
print(sfallecidos['Pichincha', 'mayo'])

Se pueden especificar valores únicamente para los primeros niveles del multi-índice, en cuyo caso el resultado es una serie indexada por los niveles restantes. Esto se conoce como *indexación parcial*:

In [ ]:
# número de fallecidos en Manabí
print(sfallecidos['Manabí'])

No es posible realizar indexación parcial por los últimos niveles directamente:

In [ ]:
# esto produce un error:
print(sfallecidos['mayo'])

Ciertas operaciones requieren que los valores de los diferentes niveles de un multi-índice se encuentren ordenados (lexicográficamente). Para ello, puede utilizarse el método `sort_index`:

In [ ]:
print(sfallecidos)
sfallecidos = sfallecidos.sort_index()
print('---')
print(sfallecidos)


Cuando el multi-índice está ordenado, se puede utilizar indexación parcial sobre cualquier nivel. Para ello, se emplea el operador de rango `:` de manera similar a como se utilizaría con arreglos multidimensionales en `numpy`. Notar que la respuesta es una serie indexada por los niveles restantes del multi-índice:

In [ ]:
# listar todos los fallecidos en mayo
print(sfallecidos[:,'mayo'])

Si el multi-índice está ordenado, es posible además la selección parcial (*partial slicing*) utilizando el operador de rango `:` sobre uno o más niveles del índice:

In [ ]:
# listar los fallecidos en Manabí y Pichincha
print(sfallecidos['Manabí': 'Pichincha'])

La indexación a través de expresiones booleanas (filtrado) también está disponible en series con multi-índices:

In [ ]:
# seleccionar todas las entradas con más de 1500 fallecidos
print(sfallecidos[sfallecidos > 1500])

Finalmente, en una selección es posible especificar listas de valores para los primeros niveles del multi-índice:

In [ ]:
# seleccionar los datos de Guayas y Pichincha
print(sfallecidos[['Guayas', 'Pichincha']])

### DataFrames con multi-índices

Consideremos nuevamente el DataFrame con multi-índices que almacena información acerca del número de nacimientos y fallecimientos en los meses de abril y mayo de los años 2019 y 2020, en las provincias de Guayas, Pichincha y Manabí:

In [ ]:
# crear multi-índice para usar en las filas
indice = pd.MultiIndex.from_product([['Guayas', 'Pichincha', 'Manabí'], ['abril', 'mayo']], 
                                  names=['Provincia', 'Mes'])
# crear multi-índice para usar en las columnas
columnas = pd.MultiIndex.from_product([['2019', '2020'], ['Nacimientos', 'Fallecimientos']], 
                                      names=['Año', 'Indicador'])
# crear DataFrame con datos inicializados a cero
df = pd.DataFrame(np.zeros((6,4), dtype=int), index= indice, columns= columnas)
# llenar datos por columnas
df['2019', 'Nacimientos'] = [6190, 6175, 3975, 4003, 2342, 2327]  # datos ficticios
df['2019', 'Fallecimientos'] = [1863, 1705, 1002, 1074, 554, 450]
df['2020', 'Nacimientos'] = [6201, 6187, 3960, 3991, 2353, 2336]  # datos ficticios
df['2020', 'Fallecimientos'] = [12242, 2580, 1284, 1582, 1654, 1257]
display(df)

La sintaxis empleada para las series puede aplicarse para seleccionar *columnas* del DataFrame, o para realizar indexación parcial por los primeros niveles de las columnas:

In [ ]:
# seleccionar columna de los nacimientos en 2020
print(df['2020', 'Nacimientos'])

# seleccionar DataFrame con la información del 2020
display(df['2020'])


El método `sort_index` ordena lexicográficamente los multi-índices de fila y de columna de un DataFrame:

In [ ]:
# ordeno el índice de las filas
df = df.sort_index()
# ordeno el índice de las columnas
df = df.sort_index(axis=1)
display(df)


Pueden usarse los métodos `loc` y `iloc` para seleccionar regiones del DataFrame de la misma manera en la se usan en el caso de índices simples:

In [ ]:
# primeras cuatro filas y últimas dos columnas:
display(df.iloc[:4,-2:])

# útlimas tres filas
display(df.iloc[3:,:])

# primera columna
display(df.iloc[:,0])

Si se emplea el método `loc`, debe recordarse que los valores de un multi-índice son tuplas:

In [ ]:
# datos de Guayas y Manabí en el 2020
display(df.loc[('Guayas', 'abril') : ('Manabí', 'mayo'), ('2020' , 'Fallecimientos'):])

Aunque el operador de rango `:` puede emplearse *entre* tuplas al utilizar `loc`, no es posible usar este operador *dentro de* una tupla. Para establecer rangos dentro de cada tupla, debe emplearse un objeto tipo `IndexSlice`:

In [ ]:
idx = pd.IndexSlice
# datos de fallecimientos
display(df.loc[:, idx[:,'Fallecimientos']])

# datos de abril
display(df.loc[idx[:, 'abril'], :])

# datos de fallecimientos en abril
display(df.loc[idx[:, 'abril'], idx[:,'Fallecimientos']])



## Reorganizando multi-índices

Es posible pasar índices de filas a columnas empleando el método `unstack`. Por defecto, el último nivel del índice de fila pasa a ser el último nivel del índice de columna. Esto puede cambiarse especificando el nivel del índice de fila a pasar a las columnas en el parámetro `level`:  

In [ ]:
display(df)

# pasar mes a índice de columna
display(df.unstack())

# pasar provincia a índice de columna
display(df.unstack(level = 0))

# pasar mes y luego provincia a índice de columna
display(df.unstack().unstack())

# pasar provincia y luego mes a índice de columna 
display(df.unstack(level = 0).unstack())


Notar que en los dos últimos ejemplos el índice de fila se queda vacío, por lo que el objeto DataFrame se transforma en una serie.

El método `stack()` realiza la operación inversa: pasa un nivel de los índices de columnas como último nivel de los índices de filas. Por defecto, se pasa el último nivel, aunque esto puede cambiarse especificando un valor para el parámetro `level`:

In [ ]:
display(df)

# pasar el indicador como último nivel del índice de fila
display(df.stack())

# pasar el año como último nivel del índice de fila
display(df.stack(level = 0))

# pasar el indicador y luego el año como últimos niveles del índice de fila
display(df.stack().stack())

# pasar el año y luego el indicador como últimos niveles del índice de fila
display(df.stack(level = 0).stack())


Notar nuevamente que en los dos últimos ejemplos el DataFrame se convierte en una serie, pues los índices de columna desaparecen.

El método `reset_index` elimina todos los niveles del índice de filas de un DataFrame (o todos los niveles del índice de una serie) y los coloca como columnas en un DataFrame. Combinando este método con el método `stack()`, es posible poner el DataFrame de nuestro ejemplo en una forma "plana".  Es conveniente especificar un nombre para la única columna de datos, empleando el parámetro `name`:

In [ ]:
# transformar el DataFrame en una serie, al eliminar los índices de columna
ds = df.stack().stack()
print(ds)

df2 = ds.reset_index(name='Valor')
display(df2)

De manera inversa, con el método `set_index` es posible definir un (multi)-índice para las filas a partir de los valores de una o más columnas de un DataFrame:

In [ ]:
# Construir multi-índice con (Indicador, Año, Mes, Provincia) 
df3 = df2.set_index(['Indicador', 'Año', 'Mes', 'Provincia'])
df3 = df3.sort_index()
display(df3)

# Pasar provincia a columnas
display(df3.unstack())

# Pasar provincia y luego año a columnas
display(df3.unstack().unstack(level=1))


## Métodos de agregación

Los métodos de agregación (como `sum`, `min`, `max`, etc) aceptan un parámetro `level` que puede utilizarse en el caso de multi-índices para especificar un nivel de agregación:

In [ ]:
df4 = df3.unstack().unstack(level=1)
display(df4)

# promedios de fallecimientos y nacimientos en abril y mayo
display(df4.mean(level='Indicador'))

# total de fallecimientos y nacimientos por mes y año
display(df4.sum(axis=1, level='Año').sum(level='Indicador'))

Más información sobre indexación jerárquica y multi-índices está disponible en la documentación del sitio web de `pandas`: <https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html>.